<a href="https://colab.research.google.com/github/BuiChiCuongs/BAOCAO_AI/blob/main/AI_Kind_of_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import library
import pandas as pd
import numpy as np
from os import listdir
from numpy import asanyarray, save
from keras.utils import load_img, img_to_array
from numpy.core.multiarray import asarray
from google.colab import drive
from keras.utils import to_categorical
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
folder = '/content/drive/MyDrive/Colab Notebooks/Picture_Train/'
photos, labels = list(), list()


In [3]:
for file in listdir(folder):
  output = 0.0
  if file.startswith('Nha'):
    output = 1.0
  if file.startswith('Cuong'):
    output = 2.0
  if file.startswith('Phuc'):
    output = 3.0
  if file.startswith('Tan'):
    output = 4.0
  photo = load_img(folder+file, target_size = (30,40))
  photo = img_to_array(photo)
  photos.append(photo)
  labels.append(output)

In [4]:
photos = asarray(photos)
labels = asarray(labels)
print(photos.shape, labels.shape)

(239, 30, 40, 3) (239,)


In [5]:
save('/content/drive/MyDrive/Colab Notebooks/Model/Face_photos.npy', photos)
save('/content/drive/MyDrive/Colab Notebooks/Model/Face_labels.npy', labels)

In [6]:
# Read data
photos = np.load('/content/drive/MyDrive/Colab Notebooks/Model/Face_photos.npy')
labels = np.load('/content/drive/MyDrive/Colab Notebooks/Model/Face_labels.npy')

In [7]:
X_train, X_test, y_train, y_test = train_test_split(photos, labels, test_size=0.2, random_state = 10)
X = X_test
X_train.shape
y_train = to_categorical(y_train, num_classes=5)

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Normalization, LeakyReLU
from keras.optimizers import Adam
from keras.layers.serialization import activation
from keras.losses import categorical_crossentropy

In [9]:
from keras.layers import LeakyReLU
model = Sequential()
model.add(Conv2D(32, (3, 3), strides=(1,1), padding = 'same', activation = 'linear', input_shape=(30, 40, 3)))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), strides=(1,1), padding = 'same', activation = 'linear'))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), strides=(1,1), padding = 'same', activation = 'linear'))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha = 0.1))
model.add(Dense(5, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 40, 32)        896       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 30, 40, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 20, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 20, 64)        18496     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 15, 20, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 10, 64)        0         
 2D)                                                    

In [10]:
# Train model
train = model.fit(X_train, y_train, batch_size = 25, epochs = 5, verbose = 1)

Epoch 1/5
8/8 [==============================] - 2s 121ms/step - loss: 56.9599 - accuracy: 0.3351
Epoch 2/5
8/8 [==============================] - 1s 118ms/step - loss: 3.2053 - accuracy: 0.6702
Epoch 3/5
8/8 [==============================] - 1s 107ms/step - loss: 0.1727 - accuracy: 0.9110
Epoch 4/5
8/8 [==============================] - 1s 67ms/step - loss: 0.0305 - accuracy: 0.9843
Epoch 5/5
8/8 [==============================] - 1s 65ms/step - loss: 0.0092 - accuracy: 0.9948


In [ ]:
# Mở ảnh
img = Image.open('Phuc1.JPG')

# Resize ảnh về kích thước mong muốn
IMG = img
img = img.resize((30, 40))

# Chuyển đổi ảnh sang numpy array và chuẩn hóa
img = np.asarray(img)
img = img.reshape(1,30,40,3)
img = img.astype('float32') / 255.0

plt.imshow(IMG)
plt.show()

matrix = np.argmax(model.predict(img), axis=-1)
class_names = ["None","Nhã","Cường","Phúc","Tấn"]

temp1 = matrix[0]
print("Bạn: ", class_names[temp1])